### Pulsed NMR

This notebook contains the code for the pulsed NMR system.

Load the required libraries.

In [ ]:
using Plots
plotly()

include("PulsedNMR.jl")
using .PulsedNMR

Connect to the TCP server running on the Red Pitaya board.

In [ ]:
c = PulsedNMR.Client()
PulsedNMR.connect!(c, "192.168.1.100")

Set frequencies, ADC sample rate and CIC decimation rate.

In [ ]:
set_freqs!(c; tx=1e6, rx=1e6)

set_rates!(c; adc=122.88, cic=48)

Define pulse sequence.

In [ ]:
d = 97  # initial read delay
u = 10  # unblank delay
a = 10  # a pulse length
b = 20  # b pulse length
ab = 90  # ab delay
bb = 180  # bb delay

clear_events!(c; read_delay=d)

# sync
add_event!(c, 1; sync=1)
# unblank delay
add_event!(c, u; gate=1, level=0, read=1)
# a pulse
add_event!(c, a; gate=1, level=100, read=1)
# ab delay
add_event!(c, ab - u; read=1)
# unblank delay
add_event!(c, u; gate=1, level=0, read=1)

for _ in 1:4
    # b pulse
    add_event!(c, b; gate=1, level=100, read=1)
    # bu delay
    add_event!(c, bb - u; read=1)
    # unblank delay
    add_event!(c, u; gate=1, level=0, read=1)
end

Run one pulse sequence.

In [ ]:
data = read_data!(c)
time = read_time!(c)

in1 = data[1:2:end]
in2 = data[2:2:end]

nothing

Plot signal.

In [ ]:
plot(time, abs.(in1), size=(1200, 400), xlabel="time, us", legend=false, tickfontsize=10)

Disconnect from the TCP server.

In [ ]:
PulsedNMR.disconnect!(c)